![JohnSnowLabs](https://nlp.johnsnowlabs.com/assets/images/logo.png)

[![Open In Colab](https://colab.research.google.com/assets/colab-badge.svg)](https://colab.research.google.com/github/JohnSnowLabs/spark-nlp-workshop/blob/master/tutorials/streamlit_notebooks/healthcare_jsl/ER_ICD10_GM_DE.ipynb)

# `sbertresolve_icd10gm` **Models**

This model maps clinical findings to their corresponding ICD-10-GM code in healthcare records using Entity Resolvers.

## 1. Colab Setup

In [ ]:
# Install the johnsnowlabs library to access Spark-OCR and Spark-NLP for Healthcare, Finance, and Legal.
! pip install -q johnsnowlabs

In [ ]:
from google.colab import files
print("Please Upload your John Snow Labs License using the button below")
license_keys = files.upload()

In [ ]:
from johnsnowlabs import *

# After uploading your license run this to install all licensed Python Wheels and pre-download Jars the Spark Session JVM
# Make sure to restart your notebook afterwards for changes to take effect

jsl.install()

## 2. Start Session

In [ ]:
from johnsnowlabs import *
# Automatically load license data and start a session with all jars user has access to
spark = jsl.start()

## 3. Select the model and construct the pipeline

In [ ]:
MODEL_NAME = ["sbertresolve_icd10gm "]

**Create the pipeline**

In [ ]:
document_assembler = nlp.DocumentAssembler()\
    .setInputCol("text")\
    .setOutputCol("document")

sentence_detector = nlp.SentenceDetector() \
    .setInputCols(['document'])\
    .setOutputCol('sentence')

tokenizer = nlp.Tokenizer()\
    .setInputCols(["sentence"])\
    .setOutputCol("token")

embeddings = nlp.WordEmbeddingsModel.pretrained('w2v_cc_300d','de', 'clinical/models') \
    .setInputCols(["sentence", 'token'])\
    .setOutputCol("embeddings")

ner = medical.NerModel.pretrained('ner_healthcare','de', 'clinical/models') \
    .setInputCols(["sentence", "token", "embeddings"]) \
    .setOutputCol("ner")

ner_converter = medical.NerConverterInternal() \
    .setInputCols(["sentence", "token", "ner"]) \
    .setOutputCol("ner_chunk")\

c2doc = nlp.Chunk2Doc()\
    .setInputCols("ner_chunk")\
    .setOutputCol("ner_chunk_doc") 

sbert_embedder = nlp.BertSentenceEmbeddings.pretrained("sent_bert_base_cased", "de")\
    .setInputCols(["ner_chunk_doc"])\
    .setOutputCol("sbert_embeddings")

icd10gm_resolver = medical.SentenceEntityResolverModel.pretrained("sbertresolve_icd10gm", "de", "clinical/models") \
    .setInputCols(["ner_chunk", "sbert_embeddings"]) \
    .setOutputCol("icd10gm_code")


nlp_pipeline = Pipeline(
    stages = [
        document_assembler,
        sentence_detector,
        tokenizer,
        embeddings,
        ner,
        ner_converter,
        c2doc,
        sbert_embedder,
        icd10gm_resolver
        ])


w2v_cc_300d download started this may take some time.
Approximate size to download 1.2 GB
[OK!]
ner_healthcare download started this may take some time.
[OK!]
sent_bert_base_cased download started this may take some time.
Approximate size to download 390.2 MB
[OK!]
sbertresolve_icd10gm download started this may take some time.
[OK!]


## 4. Create example inputs

In [ ]:
sample_text = [
"""Hallo. Ich habe folgendes Problem. Ich leide seit Jahren unter chronischen polypen. Ich habe sie mir bisher schon einmal operativ entfernen lassen, nach Monaten waren sie wieder da. Da ich seit gut einem Jahr auch nichts mehr riechen und schmecken kann, suche ich mach einer Lösung. Jetzt war ich beim Spezialisten und habe mir sagen lassen ,dass es eine neue Methode und zwar mit der Antikörper Therapie sollen die polypen unterdrückt werden bzw. Sie sollen sich zurück bilden. Hatte da jemand Erfahrungen mit gemacht? Wie läuft das genau ab? Muss ich mir dann mein Leben lang antikörper spritzen lassen? Ich war beim Arzt zu aufgeregt um alles zu fragen.
@katharina. finde es einfadh nur heftig das der arzt gleich mit einer 100 er dosis anfängt , man sollte doch langsam steigern, und das allmälich.. ich selbst hatte anfangs einen tsh von 3,75 jetzt bin ich bei 0,87 also voll in der norm. und ich nehm gerade mal 50µ.... vllt solltestd u ml zu einem anderen artz gehen???
ich will dir nur aufzeigen, dass gegenseitiges verständnis auch ein stück weit dazu führen kann, dass die ganze beziehung zwischen euch drei etwas weniger anstrengend für ihn wird. und ja, das ist doof, weil er kein verständnis für euch aufbringt, ihr aber für ihn. aber so ist das leben mit einem autisten nunmal - er hat sich seinen autismus ebenso wenig ausgesucht, wie ihr.""",

"""Nach der Einnahme von Symbioflor2 (1992) entwickelte sich dann dieser Fäkalgeruch bei mir, den ich seit dem nicht mehr losgeworden bin. Vom Symbioflor2 bekam ich dann auch immer stärkere körperliche Beschwerden, so daß ich es mit der Angst zu tun bekam und meinen Arzt bat mir Cotrim zuverschreiben (das zweitemal nach 1983). Die Folge war leider, daß ich starken Fußschweiß bekam mit entsprechendem Geruch. Auch meine Hände fingen stark an zu riechen.
Ich war im Schlaflabor. Nur eine Auffälligkeit. In der ersten Nacht dort trat ich verfrüht in den Traumschlaf ein, was ein Kriterium für Narkolepsie ist. Bluttest und Gentest aber waren negativ. Meine Lungen sind super, mein MRT unauffällig. Jetzt bin ich in einer neurologischen Spezialklinik. Der Arzt meinte, dass eine sehr geringe Wahrscheinlichkeit besteht, dass ich doch an Narkolepsie leide. Ich soll in einem halben Jahr nochmal dort ins Schlaflabor. Medikamente gegen die Müdigkeit will er mir nicht verschreiben. Antidepressiva wären es, aber die müsste ich mir abgewöhnen, bevor ich in das Schlaflabor gehe. Also lieber "clean" bleiben.
Ich glaube mittlerweile, dass das Problem ganz woanders liegt... obwohl ich als Kind laut Aussagen meiner Eltern immer recht schnell dabei war, was laufen, Fahrradfahren etc. angeht, habe ich jetzt gerade als Erwachsener gemerkt, dass ich sehr langsam dabei bin, komplexe Bewegungen einzulernen und richtig auszuführen, sei es z.B. Tanzen oder Kampfsport. Wobei das sicherlich was ist, was mit der Zeit und Training schon deutlich besser geworden ist.""",

"""Ja das ist bekannt gewesen. Und die haben sich danach gerichtet. Hatte das Gefühl das er sich schon sehr wohl da gefühlt. Haben das auch von den Betreuern als Rückmeldung erhalten. Er ist zwar immer furchtbar aufgeregt wenn es dann los geht, aber wenn er dann da ist, freut er sich doch. Müssen zwar dann auch meist er sehr lange Autofahrt in Kauf nehmen da es das nicht in unserer Gegend gibt. Aber wenn er wenigstens daran eine Freude, warum nicht.
Leute, lest euch diesen Beitrag vollständig durch! Cosma und ich, wir haben unsere Erlebnisse gut dokumentiert. Wir waren am Anfang genauso ratlos, wie du (und noch andere, die hier dieselben Fragen stellem), die Diagnose ist für solche Sachen entweder Depressionen oder Burnout bzw. beides (wenn nichts Körperliches vorliegt). Manche haben noch zusätzlich Angstzustände oder Panikattacken. Die Diagnose sollte allerdings ein Neurologe/Psychiater stellen.""",

"""Das ist doch immer eine Überlegung wert. Bis vor kurzem war das sogar noch die Strategie ganzer Länder, u. a. GB. Also ich bin zum Besispiel einer von den süßen Opis, die derzeit bloß nicht an den Eiern gedrückt werden sollen. Ich würde mir tatsächlich eine Ansteckung mit einem einigermaßen moderaten Verlauf wünschen. Dann hätte ich, nach allem was man weiß, u. a. von Rhesusaffen, eine Grundimmunität und könnte rausgehen und meinen Mitmenschen unter die Arme greifen. Ich bin halt alt, aber net unbedingt schwach. Rüstiger Greis, sozusagen"""]


In [ ]:
from pyspark.sql.types import StringType, IntegerType

df = spark.createDataFrame(sample_text,StringType()).toDF('text')

df.show(truncate = 100)

+----------------------------------------------------------------------------------------------------+
|                                                                                                text|
+----------------------------------------------------------------------------------------------------+
|Hallo. Ich habe folgendes Problem. Ich leide seit Jahren unter chronischen polypen. Ich habe sie ...|
|Nach der Einnahme von Symbioflor2 (1992) entwickelte sich dann dieser Fäkalgeruch bei mir, den ic...|
|Ja das ist bekannt gewesen. Und die haben sich danach gerichtet. Hatte das Gefühl das er sich sch...|
|Das ist doch immer eine Überlegung wert. Bis vor kurzem war das sogar noch die Strategie ganzer L...|
+----------------------------------------------------------------------------------------------------+



## 5. Use the pipeline to create outputs

In [ ]:
# returns spark df resolution results

def get_codes_from_df(result_df, chunk, output_col, hcc= False):
    
    
    if hcc:
        
        df = result_df.select(F.explode(F.arrays_zip(result_df[chunk].result, 
                                                     result_df[chunk].metadata, 
                                                     result_df[output_col].result, 
                                                     result_df[output_col].metadata)).alias("cols")) \
                      .select(F.expr("cols['1']['sentence']").alias("sent_id"),
                              F.expr("cols['0']").alias("ner_chunk"),
                              F.expr("cols['1']['entity']").alias("entity"), 
                              F.expr("cols['2']").alias("icd10_code"),
                              F.expr("cols['3']['all_k_results']").alias("all_codes"),
                              F.expr("cols['3']['all_k_resolutions']").alias("resolutions"),
                              F.expr("cols['3']['all_k_aux_labels']").alias("hcc_list")).toPandas()



        codes = []
        resolutions = []
        hcc_all = []

        for code, resolution, hcc in zip(df['all_codes'], df['resolutions'], df['hcc_list']):

            codes.append(code.split(':::'))
            resolutions.append(resolution.split(':::'))
            hcc_all.append(hcc.split(":::"))

        df['all_codes'] = codes  
        df['resolutions'] = resolutions
        df['hcc_list'] = hcc_all
        
    else:
                       
        df = result_df.select(F.explode(F.arrays_zip(result_df[chunk].result, 
                                                     result_df[chunk].metadata, 
                                                     result_df[output_col].result, 
                                                     result_df[output_col].metadata)).alias("cols")) \
                      .select(F.expr("cols['1']['sentence']").alias("sent_id"),
                              F.expr("cols['0']").alias("ner_chunk"),
                              F.expr("cols['1']['entity']").alias("entity"), 
                              F.expr("cols['2']").alias(f"{output_col}"),
                              F.expr("cols['3']['all_k_results']").alias("all_codes"),
                              F.expr("cols['3']['all_k_resolutions']").alias("resolutions")).toPandas()



        codes = []
        resolutions = []

        for code, resolution in zip(df['all_codes'], df['resolutions']):

            codes.append(code.split(':::'))
            resolutions.append(resolution.split(':::'))

        df['all_codes'] = codes  
        df['resolutions'] = resolutions
        
    
    return df

In [ ]:
result = nlp_pipeline.fit(df).transform(df)

result.select(F.explode(F.arrays_zip(result.ner_chunk.result, 
                                     result.ner_chunk.begin, 
                                     result.ner_chunk.end,
                                     result.ner_chunk.metadata,
                                     result.icd10gm_code.result,
                                     result.icd10gm_code.metadata,)).alias("cols"))\
      .select(F.expr("cols['0']").alias("chunk"),
              F.expr("cols['1']").alias("begin"),
              F.expr("cols['2']").alias("end"),
              F.expr("cols['3']['entity']").alias("entity"),
              F.expr("cols['4']").alias("icd10_code"),
              F.expr("cols['5']['all_k_results']").alias("all_codes"),
              F.expr("cols['5']['all_k_resolutions']").alias("resolutions")).show(truncate=30)

+-------------------+-----+---+--------------------+----------+------------------------------+------------------------------+
|              chunk|begin|end|              entity|icd10_code|                     all_codes|                   resolutions|
+-------------------+-----+---+--------------------+----------+------------------------------+------------------------------+
|        seit Jahren|   45| 55|    TIME_INFORMATION|     M83.8|M83.8:::T89.02:::S92.28:::M...|Sonstige Osteomalazie im Er...|
|chronischen polypen|   63| 81|   MEDICAL_CONDITION|       I83|I83:::Z94.7:::Z94.5:::Z00.3...|Varizen der unteren Extremi...|
|           operativ|  121|128|      MEDICAL_DEVICE|     B35.8|B35.8:::I46:::I50:::Z30:::Z...|Sonstige Dermatophytosen:::...|
|          entfernen|  130|138|           TREATMENT|    R50.80|R50.80:::R09.2:::I46.0:::O8...|Fieber unbekannter Ursache:...|
|       nach Monaten|  148|159|    TIME_INFORMATION|     M83.8|M83.8:::P37.4:::M80.8:::T90...|Sonstige Osteomalazie im

## 6. Visualize results

In [ ]:
from sparknlp_display import EntityResolverVisualizer

resolver_viz = EntityResolverVisualizer()


for j in range(len(sample_text)):
    resolver_viz.display(result = result.collect()[j], label_col = "ner_chunk", resolution_col="icd10gm_code")
    print("\n\n")